In [2]:
import numpy as np
import random
import sys

In [32]:
#Let X be an n x p matrix of data and y be and n-D vector
n = 100
p = 10
X = np.zeros((n, p))
y = np.ones(n)

In [45]:
#Define the mean vector (all zeros)
mean = np.zeros(p)
mean2 = np.zeros(n)

# Define the covariance matrix (identity matrix)
covariance = np.identity(p)
covariance2 = np.identity(n)

# Generate a sample from the multivariate normal distribution
X = np.random.multivariate_normal(mean, covariance, size=n)
err = np.random.normal(0, 1, size=(n))
beta = np.zeros(p)
beta[0] = 2
beta[1] = 5

# print(X.shape)
# print(beta.shape) # should be p
# print(err.shape)
y = X @ beta # + err

In [34]:
#set your regularization parameter >= 0
lmbda = 0.5

In [5]:
#expression to minimize (print)
beta = np.zeros(p)
expr = 0.5 / n * np.linalg.norm(y - np.matmul(X, beta))**2 + lmbda * np.linalg.norm(beta, ord = 1)

KKT Conditions to find the minimum

In [6]:
#subgradient
def s(beta, j):
  if beta[j] != 0:
    return np.sign(beta[j])
  return (2 * random.random() - 1) #is this right? how to we do calculations with the subgradient?

In [7]:
#equations (print)
for j in range(n):
  -1/n * np.dot(X[:,j], y - np.matmul(X, beta)) + lmbda * s(beta, j)

Calculate solutions using closed form

In [8]:
#soft-thresholding function
#y, z are N-d vectors
def soft_threshold(y, z, lmbda):
  N = y.shape

  val = (y @ z) / N

  if val > lmbda:
    return (val - lmbda)
  elif val < -lmbda:
    return (val + lmbda)
  return 0


In [9]:
#x, y are N-d vectors
def LASSO_one_pred(y, x, lmbda):
  return soft_threshold(y, x, lmbda)

In [46]:
#y is N-d vectors
#X is N x k matrix
def LASSO_multi_pred(y, X, lmbda):
  #get data set dimensions
  N, K = X.shape

  # use single predictor soft thresholding if one predictor only
  if K == 1:
    LASSO_one_pred(y, X, lmbda)

  # multiple predictors coordinate descent
  beta_hat = np.zeros(K)
  beta_prev = np.zeros(K)
  dif = 10000000000
  iter = 0

  # deal with starting dist how?
  while dif > 1e-6:
    for j in range(K):
      rj = np.zeros(N) #partial residual
      for i in range(N):
        #rj[i] = y_i - sum_{k != j} x_ik * beta_hat_k
        sum = np.dot(X[i], beta_hat) # sum_{k = 1}^K x_ik * beta_hat_k
        rj[i] = y[i] - sum + X[i,j] * beta_hat[j]

      beta_hat[j] = soft_threshold(rj, X[:,j], lmbda) # update

    # check the stopping point
    # only check whether or not to stop once all coordinates have been updated
    dif = np.linalg.norm(beta_hat - beta_prev, ord = 1) #l1 norm
    beta_prev = beta_hat
    iter += 1

  return beta_hat, iter

In [47]:
beta_hat, iter = LASSO_multi_pred(y, X, lmbda)
print(beta_hat)
print(iter)

[1.26536727 5.00003622 0.         0.         0.         0.
 0.         0.         0.         0.        ]
2


/var/folders/j4/hm50bg4n6vv8hhf19l5sjhsr0000gr/T/ipykernel_48188/753243011.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  beta_hat[j] = soft_threshold(rj, X[:,j], lmbda) # update


In [48]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Get the slope (coefficient) and intercept
slope = model.coef_[0]
intercept = model.intercept_

print(f"Slope: {slope:.2f}")
print(f"Intercept: {intercept:.2f}")

Slope: 2.00
Intercept: -0.00


In [49]:
model.coef_

array([ 2.00000000e+00,  5.00000000e+00,  1.05471187e-15,  1.33226763e-15,
        7.77156117e-16,  1.33226763e-15, -1.57512892e-15, -2.66453526e-15,
       -3.27515792e-15, -2.16493490e-15])